In [1]:
pipe?

Object `pipe` not found.


In [2]:
import re

import numpy as np
import pandas as pd
from tqdm import tqdm

from dankypipe import pipe

tqdm.pandas()

def isfloat(x):
    try:
        float(x)
        return True
    except:
        return False
    
def isint(x):
    try:
        int(x)
        return True
    except:
        return False
    

In [3]:
pipe?

In [2]:
train = pd.read_pickle('train.pickle').sort_values(by='AvSigVersion')
test = pd.read_pickle('test.pickle').sort_values(by='AvSigVersion')

train = train.rename(columns={'HasDetections':'Target'})

In [3]:
def transform(df):
    df.loc[df.RtpStateBitfield.isnull(), 'RtpStateBitfield'] = 34

    df.RtpStateBitfield = [
        str(bin(int(float(c))))[2:].zfill(4)[-4:] if str(c) != 'nan' else np.nan 
        for c in tqdm(df.RtpStateBitfield, desc='converting to bit string')
    ]
    
    for i in tqdm(range(1, 5), desc='expanding features'):
        cname = f'RtpStateBitfield_{i}'
        df[cname] = df.RtpStateBitfield.apply(
            lambda x: bool(x[4-i]=='1') if str(x) != 'nan' else np.nan
        )

    return df

In [4]:
cols  = ['RtpStateBitfield', 'MachineIdentifier']
train = train[cols+['Target']]
test  = test[cols]

In [5]:
train = transform(train)
test  = transform(test)

 50%|████▉     | 4418147/8921483 [00:05<00:04, 974247.36it/s]


 25%|██▍       | 1961308/7853253 [00:02<05:53, 16658.63it/s]


expanding features: 100%|██████████| 4/4 [00:34<00:00,  8.69s/it]


In [9]:
train.head()

,RtpStateBitfield,MachineIdentifier,Target,RtpStateBitfield_1,RtpStateBitfield_2,RtpStateBitfield_3,RtpStateBitfield_4
6822125,0001,c3c4bc04dc5f1c7245a862e52634428e,0,True,False,False,False
7285638,0011,d106fcb0c6482265956c05ffbaf60744,0,True,True,False,False
5050150,0001,90eeb2d77a5f58c0afe71de24f29bb50,0,True,False,False,False
165482,0000,04c1c463cbb6e2bfae34c4c66fd3242c,0,False,False,False,False
735046,0001,151dd3600408f025207073d09cbc6d5d,0,True,False,False,False


In [10]:
test.head()

,RtpStateBitfield,MachineIdentifier,RtpStateBitfield_1,RtpStateBitfield_2,RtpStateBitfield_3,RtpStateBitfield_4
7252423,0000,ec6910b4d9e0baae203e9819227659ec,False,False,False,False
6804872,0000,ddd66992da9cbb12db76d9d874fedf8b,False,False,False,False
6882538,0000,e05db268c5f1e48e5fa63de1f39f02d7,False,False,False,False
6856130,0001,df81a38177efaac6b95df42ddef504e6,True,False,False,False
2544324,0001,52eb832b198099b467d39481a77afcef,True,False,False,False


In [11]:
val_idx = int(len(train)*.7)
val_idx

6245038

In [12]:
train = train.rename(columns={'RtpStateBitfield':'RtpStateBitfield_str'})
test  = test.rename(columns={'RtpStateBitfield':'RtpStateBitfield_str'})

pbar = tqdm(total=len(train.columns)-2)

for c in train.columns:
    if c == 'MachineIdentifier' or c == 'Target':
        continue
        
    pbar.set_description(c)
    
    train_ = train[['MachineIdentifier', c]].iloc[:val_idx, :]
    val_   = train[['MachineIdentifier', c]].iloc[val_idx:, :]
    test_  =  test[['MachineIdentifier', c]]
    
    try:
        pipe.upload_feature(c, (train_, val_, test_))
    except ValueError:
        pass
    
    pbar.update(1)

RtpStateBitfield_4: 100%|██████████| 5/5 [5:58:59<00:00, 4220.55s/it]  